In [1]:
from tensor import Tensor, Dependency, add
import numpy as np

In [2]:
t1 = Tensor([1, 2, 3], requires_grad=True)
t2 = t1.sum()
t2.backward()

In [3]:
np.ones_like([1])

array([1])

In [ ]:
isinstance(np.array([1, 1]), np.ndarray)

True

In [ ]:
def grad_fn(grad: np.ndarray) -> np.ndarray:
    """grad is necessarily a 0-tensor, so each input element contributes that much"""
    return grad * np.ones_like([3, 3, 3])

In [ ]:
t1 = Tensor([3.0, 2.0, 4.0], requires_grad=True)
t2 = Tensor([3.0, 3.0, 3.0], requires_grad=True)
t3 = Tensor([3.0, 3.0, 3.0], requires_grad=True, depends_on=[Dependency(t1, grad_fn)])
t4 = Tensor([3.0, 3.0, 3.0], requires_grad=True, depends_on=[Dependency(t2, grad_fn)])
t5 = Tensor(
    [3.0, 3.0, 3.0],
    requires_grad=True,
    depends_on=[Dependency(t3, grad_fn), Dependency(t4, grad_fn)],
)
t6 = Tensor(
    [3.0, 3.0, 3.0],
    requires_grad=True,
    depends_on=[Dependency(t2, grad_fn), Dependency(t5, grad_fn)],
)
t7 = Tensor(
    [3.0, 3.0, 3.0],
    requires_grad=True,
    depends_on=[Dependency(t6, grad_fn), Dependency(t5, grad_fn)],
)
t7.backward(Tensor([9, 9, 9]))


In [29]:
t5.grad

Tensor([18. 18. 18.], requires_grad=False)

In [13]:
t7.data.sum()

np.float64(9.0)

In [2]:
t1 = Tensor(np.array([[1], [2]]), requires_grad=True)  # Shape (2,1)
t2 = Tensor(np.array([[3, 4], [5, 6]]), requires_grad=True)  # Shape (2,2)

result = add(t1, t2)
result

Tensor([[4 5]
 [7 8]], requires_grad=True)

In [9]:
t1 = Tensor([1, 2, 3], requires_grad=True)
t2 = Tensor(10, requires_grad=True)  # Scalar
t3 = add(t1, t2)
t3.backward(Tensor([1, 1, 1]))  # Passing a gradient of [1, 1, 1]

t2.grad.data.tolist()

3.0

In [ ]:
def neg(t: Tensor) -> Tensor:
    data = -t.data
    requires_grad = t.requires_grad
    if requires_grad:
        depends_on = [Dependency(t, lambda x: -x)]
    else:
        depends_on = []
    return Tensor(data, requires_grad, depends_on)


def sub(t1: Tensor, t2: Tensor):
    data = t1.data - t2.data
    requires_grad = t1.requires_grad or t2.requires_grad
    depends_on: list[Dependency] = []
    if t1.requires_grad:

        def grad_fn1(grad: np.ndarray) -> np.ndarray:
            if t1.data.shape() == grad.shape():
                return grad
            added_dims = grad.ndim - t1.data.ndim
            for _ in range(added_dims):
                grad = grad.sum(axis=0)
            for i, dim in enumerate(t1.data.shape()):
                if dim == 1:
                    grad = grad.sum(axis=i, keepdims=True)
            return grad

        depends_on.append(Dependency(t1, grad_fn1))

    if t2.requires_grad:

        def grad_fn2(grad: np.ndarray) -> np.ndarray:
            if t2.data.shape() == grad.shape():
                return grad
            added_dims = grad.ndim - t2.data.ndim
            for _ in range(added_dims):
                grad = grad.sum(axis=0)
            for i, dim in enumerate(t2.data.shape()):
                if dim == 1:
                    grad = grad.sum(axis=i, keepdims=True)
            return grad

        depends_on.append(Dependency(t2, grad_fn2))
    return Tensor(data, requires_grad, depends_on)